In [1]:
#!/usr/bin/env python3
"""
Dự đoán Năng lượng Điện với XGBoost và DiCE

Mục tiêu:
1. Phân tích Features: Tìm ra các features quan trọng nhất (PCA, Feature Importance, Correlation)
2. Dự đoán Time Series: Train XGBoost để dự đoán điện tiêu thụ trong tương lai (time step = 1 giờ) với prediction interval
3. DiCE Counterfactual Explanations: Gợi ý điều chỉnh features để không vượt ngưỡng threshold
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb

# DiCE
import sys
import os
os.chdir('..')
import dice_ml
from dice_ml import Dice

# Visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("=" * 80)
print("DỰ ĐOÁN NĂNG LƯỢNG ĐIỆN VỚI XGBOOST VÀ DiCE")
print("=" * 80)

# ============================================================================
# PHẦN 1: LOAD VÀ XỬ LÝ DỮ LIỆU
# ============================================================================

print("\n" + "=" * 80)
print("PHẦN 1: LOAD VÀ XỬ LÝ DỮ LIỆU")
print("=" * 80)

# Đường dẫn đến các file dữ liệu
base_path = "./datasets"

electricity_path = f"{base_path}/electricity_cleaned.csv"
metadata_path = f"{base_path}/metadata.csv"
weather_path = f"{base_path}/weather.csv"

print("\nĐang load dữ liệu...")

# Load metadata
df_metadata = pd.read_csv(metadata_path)
print(f"✅ Metadata: {df_metadata.shape}")

# Load electricity data
df_electricity = pd.read_csv(electricity_path, parse_dates=['timestamp'])
print(f"✅ Electricity: {df_electricity.shape}")

# Load weather data
df_weather = pd.read_csv(weather_path, parse_dates=['timestamp'])
print(f"✅ Weather: {df_weather.shape}")

print("\n📊 Tóm tắt dữ liệu:")
print(f"  - Số buildings: {len(df_metadata)}")
print(f"  - Số timestamps: {len(df_electricity)}")
print(f"  - Khoảng thời gian: {df_electricity['timestamp'].min()} đến {df_electricity['timestamp'].max()}")

# Chuyển electricity data từ wide format sang long format
print("\nĐang chuyển đổi electricity data sang long format...")

df_electricity_long = pd.melt(
    df_electricity,
    id_vars=['timestamp'],
    var_name='building_id',
    value_name='electricity_consumption'
)

# Loại bỏ NaN
df_electricity_long = df_electricity_long.dropna(subset=['electricity_consumption'])

# Chỉ giữ lại các buildings có electricity meter
buildings_with_electricity = df_metadata[df_metadata['electricity'] == 'Yes']['building_id'].tolist()
df_electricity_long = df_electricity_long[df_electricity_long['building_id'].isin(buildings_with_electricity)]

print(f"✅ Đã chuyển đổi: {df_electricity_long.shape}")
print(f"  - Số buildings có dữ liệu: {df_electricity_long['building_id'].nunique()}")
print(f"  - Số lượng readings: {len(df_electricity_long)}")

# Merge với metadata
print("\nĐang merge với metadata...")

df_merged = pd.merge(
    df_electricity_long,
    df_metadata,
    on='building_id',
    how='inner'
)

print(f"✅ Sau khi merge: {df_merged.shape}")
print(f"  - Số buildings: {df_merged['building_id'].nunique()}")
print(f"  - Số timestamps: {df_merged['timestamp'].nunique()}")

# Merge với weather data (theo site_id và timestamp)
print("\nĐang merge với weather data...")

df_final = pd.merge(
    df_merged,
    df_weather,
    on=['timestamp', 'site_id'],
    how='left'
)

print(f"✅ Sau khi merge weather: {df_final.shape}")

# Feature Engineering: Tạo các features thời gian
print("\nĐang tạo features thời gian...")

df_final['hour'] = df_final['timestamp'].dt.hour
df_final['day_of_week'] = df_final['timestamp'].dt.dayofweek
df_final['day_of_month'] = df_final['timestamp'].dt.day
df_final['month'] = df_final['timestamp'].dt.month
df_final['is_weekend'] = (df_final['day_of_week'] >= 5).astype(int)

# Tạo season feature
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df_final['season'] = df_final['month'].apply(get_season)

# Tạo lag features (điện tiêu thụ giờ trước)
df_final = df_final.sort_values(['building_id', 'timestamp'])
df_final['electricity_lag1'] = df_final.groupby('building_id')['electricity_consumption'].shift(1)
df_final['electricity_lag24'] = df_final.groupby('building_id')['electricity_consumption'].shift(24)  # Cùng giờ ngày hôm trước

# Rolling statistics
df_final['electricity_rolling_mean_24h'] = df_final.groupby('building_id')['electricity_consumption'].transform(
    lambda x: x.rolling(window=24, min_periods=1).mean()
)
df_final['electricity_rolling_std_24h'] = df_final.groupby('building_id')['electricity_consumption'].transform(
    lambda x: x.rolling(window=24, min_periods=1).std()
)

print("✅ Đã tạo các features thời gian và lag features")
print(f"Dataset shape: {df_final.shape}")

DỰ ĐOÁN NĂNG LƯỢNG ĐIỆN VỚI XGBOOST VÀ DiCE

PHẦN 1: LOAD VÀ XỬ LÝ DỮ LIỆU

Đang load dữ liệu...
✅ Metadata: (1636, 32)
✅ Electricity: (17544, 1579)
✅ Weather: (331166, 10)

📊 Tóm tắt dữ liệu:
  - Số buildings: 1636
  - Số timestamps: 17544
  - Khoảng thời gian: 2016-01-01 00:00:00 đến 2017-12-31 23:00:00

Đang chuyển đổi electricity data sang long format...
✅ Đã chuyển đổi: (25212579, 3)
  - Số buildings có dữ liệu: 1572
  - Số lượng readings: 25212579

Đang merge với metadata...
✅ Sau khi merge: (25212579, 34)
  - Số buildings: 1572
  - Số timestamps: 17544

Đang merge với weather data...
✅ Sau khi merge weather: (25212579, 42)

Đang tạo features thời gian...
✅ Đã tạo các features thời gian và lag features
Dataset shape: (25212579, 52)


In [5]:
# ============================================================================
# PHẦN 2: PHÂN TÍCH FEATURES - TÌM FEATURES QUAN TRỌNG
# ============================================================================

print("\n" + "=" * 80)
print("PHẦN 2: PHÂN TÍCH FEATURES - TÌM FEATURES QUAN TRỌNG")
print("=" * 80)

# Chuẩn bị dữ liệu cho phân tích
# Chọn một subset buildings để phân tích nhanh hơn
np.random.seed(42)
sample_buildings = np.random.choice(
    df_final['building_id'].unique(), 
    size=min(50, df_final['building_id'].nunique()), 
    replace=False
)
df_analysis = df_final[df_final['building_id'].isin(sample_buildings)].copy()

print(f"\nĐang phân tích {len(sample_buildings)} buildings...")
print(f"Dataset shape: {df_analysis.shape}")

# Loại bỏ các dòng có quá nhiều missing values
df_analysis = df_analysis.dropna(subset=['electricity_consumption', 'sqm', 'airTemperature'])

print(f"Sau khi loại bỏ missing values: {df_analysis.shape}")

# Xác định các features để phân tích
continuous_features = [
    'sqm', 'yearbuilt', 'numberoffloors', 'occupants',
    'airTemperature', 'cloudCoverage', 'dewTemperature', 'windSpeed', 'seaLvlPressure',
    'hour', 'day_of_week', 'day_of_month', 'month',
    'electricity_lag1', 'electricity_lag24', 
    'electricity_rolling_mean_24h', 'electricity_rolling_std_24h'
]

categorical_features = [
    'primaryspaceusage', 'sub_primaryspaceusage', 'site_id', 'timezone', 'season', 'is_weekend'
]

# Loại bỏ các features không có trong dataset
continuous_features = [f for f in continuous_features if f in df_analysis.columns]
categorical_features = [f for f in categorical_features if f in df_analysis.columns]

print(f"\nContinuous features ({len(continuous_features)}): {continuous_features}")
print(f"Categorical features ({len(categorical_features)}): {categorical_features}")

# Tạo dataset cho phân tích
X_analysis = df_analysis[continuous_features + categorical_features].copy()
y_analysis = df_analysis['electricity_consumption'].copy()

# Fill missing values
for col in continuous_features:
    if col in X_analysis.columns:
        X_analysis[col] = X_analysis[col].fillna(X_analysis[col].median())

for col in categorical_features:
    if col in X_analysis.columns:
        X_analysis[col] = X_analysis[col].fillna(
            X_analysis[col].mode()[0] if len(X_analysis[col].mode()) > 0 else 'Unknown'
        )

# Encode categorical features
label_encoders = {}
for col in categorical_features:
    if col in X_analysis.columns:
        le = LabelEncoder()
        X_analysis[col] = le.fit_transform(X_analysis[col].astype(str))
        label_encoders[col] = le

print(f"\n✅ Dataset cho phân tích: {X_analysis.shape}")
print(f"Target: {y_analysis.shape}")

# 2.1. Correlation Analysis
print("\n" + "-" * 80)
print("2.1. Correlation Analysis")
print("-" * 80)

correlation_data = X_analysis[continuous_features].copy()
correlation_data['electricity_consumption'] = y_analysis

correlations = correlation_data.corr()['electricity_consumption'].sort_values(ascending=False)
correlations = correlations.drop('electricity_consumption')

print("\n📊 Correlation với Electricity Consumption:")
print("=" * 60)
for feature, corr in correlations.items():
    print(f"{feature:30s}: {corr:7.4f}")

print(f"\n✅ Top 5 features có correlation cao nhất:")
print(correlations.head(5))

# Visualization
plt.figure(figsize=(10, 8))
correlations.plot(kind='barh')
plt.title('Correlation với Electricity Consumption', fontsize=14, fontweight='bold')
plt.xlabel('Correlation Coefficient', fontsize=12)
plt.tight_layout()
plt.savefig('./analysis/correlation_analysis.png', dpi=150, bbox_inches='tight')
print("\n✅ Đã lưu biểu đồ correlation vào: correlation_analysis.png")
plt.close()

# 2.2. Feature Importance với XGBoost
print("\n" + "-" * 80)
print("2.2. Feature Importance với XGBoost")
print("-" * 80)

print("Đang train XGBoost để lấy feature importance...")

# Chia train/test theo thời gian
df_analysis_sorted = df_analysis.sort_values('timestamp')
split_idx = int(len(df_analysis_sorted) * 0.8)

X_train_imp = X_analysis.iloc[:split_idx]
y_train_imp = y_analysis.iloc[:split_idx]
X_test_imp = X_analysis.iloc[split_idx:]
y_test_imp = y_analysis.iloc[split_idx:]

# Train XGBoost
xgb_model_imp = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1
)

xgb_model_imp.fit(X_train_imp, y_train_imp)

# Lấy feature importance
feature_importance = pd.DataFrame({
    'feature': X_analysis.columns,
    'importance': xgb_model_imp.feature_importances_
}).sort_values('importance', ascending=False)

print("\n📊 Feature Importance (XGBoost):")
print("=" * 60)
for idx, row in feature_importance.iterrows():
    print(f"{row['feature']:30s}: {row['importance']:7.4f}")

print(f"\n✅ Top 10 features quan trọng nhất:")
print(feature_importance.head(10))

# Visualization
plt.figure(figsize=(10, 10))
feature_importance.plot(x='feature', y='importance', kind='barh', figsize=(10, 10))
plt.title('Feature Importance (XGBoost)', fontsize=14, fontweight='bold')
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout()
plt.savefig('./analysis/feature_importance.png', dpi=150, bbox_inches='tight')
print("\n✅ Đã lưu biểu đồ feature importance vào: feature_importance.png")
plt.close()

# 2.3. PCA Analysis
print("\n" + "-" * 80)
print("2.3. PCA Analysis")
print("-" * 80)

print("Đang thực hiện PCA...")

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_analysis[continuous_features])

# PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Tính explained variance
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print(f"\n📊 PCA Results:")
print(f"  - Số components: {len(explained_variance)}")
print(f"  - Explained variance của 5 components đầu: {explained_variance[:5]}")
print(f"  - Cumulative variance của 5 components đầu: {cumulative_variance[:5]}")

# Tìm số components cần để giải thích 90% variance
n_components_90 = np.where(cumulative_variance >= 0.90)[0][0] + 1
print(f"\n  - Số components để giải thích 90% variance: {n_components_90}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Explained variance
axes[0].plot(range(1, len(explained_variance) + 1), explained_variance, 'bo-')
axes[0].set_xlabel('Principal Component', fontsize=12)
axes[0].set_ylabel('Explained Variance Ratio', fontsize=12)
axes[0].set_title('Explained Variance by Component', fontsize=14, fontweight='bold')
axes[0].grid(True)

# Cumulative variance
axes[1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'ro-')
axes[1].axhline(y=0.90, color='g', linestyle='--', label='90% Variance')
axes[1].axvline(x=n_components_90, color='g', linestyle='--')
axes[1].set_xlabel('Number of Components', fontsize=12)
axes[1].set_ylabel('Cumulative Explained Variance', fontsize=12)
axes[1].set_title('Cumulative Explained Variance', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('./analysis/pca_analysis.png', dpi=150, bbox_inches='tight')
print("\n✅ Đã lưu biểu đồ PCA vào: pca_analysis.png")
plt.close()

# Component loadings cho PC1 và PC2
pc1_loadings = pd.DataFrame({
    'feature': continuous_features,
    'PC1': pca.components_[0],
    'PC2': pca.components_[1]
}).sort_values('PC1', key=abs, ascending=False)

print(f"\n📊 Top 10 features đóng góp nhiều nhất vào PC1:")
print(pc1_loadings.head(10)[['feature', 'PC1']])

# 2.4. Tổng hợp Features Quan trọng
print("\n" + "-" * 80)
print("2.4. Tổng hợp Features Quan trọng")
print("-" * 80)

# Chọn features quan trọng dựa trên phân tích
selected_continuous_features = [
    'sqm',                    # Diện tích - rất quan trọng
    'occupants',              # Số người - rất quan trọng
    'yearbuilt',              # Năm xây dựng
    'numberoffloors',         # Số tầng
    'airTemperature',         # Nhiệt độ - rất quan trọng
    'hour',                   # Giờ trong ngày - pattern sử dụng
    'day_of_week',            # Ngày trong tuần
    'month',                  # Tháng - mùa
    'electricity_lag1',       # Lag 1 giờ
    'electricity_lag24',      # Lag 24 giờ (cùng giờ hôm trước)
    'electricity_rolling_mean_24h',  # Trung bình 24h
    'cloudCoverage',          # Độ che phủ mây
    'windSpeed'               # Tốc độ gió
]

selected_categorical_features = [
    'primaryspaceusage',      # Loại sử dụng - rất quan trọng
    'site_id',                # Site
    'timezone',               # Múi giờ
    'season',                 # Mùa
    'is_weekend'              # Cuối tuần
]

# Loại bỏ các features không có trong dataset
selected_continuous_features = [f for f in selected_continuous_features if f in df_final.columns]
selected_categorical_features = [f for f in selected_categorical_features if f in df_final.columns]

print("\n✅ Selected Features:")
print(f"\nContinuous features ({len(selected_continuous_features)}):")
for f in selected_continuous_features:
    print(f"  - {f}")

print(f"\nCategorical features ({len(selected_categorical_features)}):")
for f in selected_categorical_features:
    print(f"  - {f}")

print(f"\nTổng cộng: {len(selected_continuous_features) + len(selected_categorical_features)} features")


PHẦN 2: PHÂN TÍCH FEATURES - TÌM FEATURES QUAN TRỌNG

Đang phân tích 50 buildings...
Dataset shape: (806777, 52)
Sau khi loại bỏ missing values: (804216, 52)

Continuous features (17): ['sqm', 'yearbuilt', 'numberoffloors', 'occupants', 'airTemperature', 'cloudCoverage', 'dewTemperature', 'windSpeed', 'seaLvlPressure', 'hour', 'day_of_week', 'day_of_month', 'month', 'electricity_lag1', 'electricity_lag24', 'electricity_rolling_mean_24h', 'electricity_rolling_std_24h']
Categorical features (6): ['primaryspaceusage', 'sub_primaryspaceusage', 'site_id', 'timezone', 'season', 'is_weekend']

✅ Dataset cho phân tích: (804216, 23)
Target: (804216,)

--------------------------------------------------------------------------------
2.1. Correlation Analysis
--------------------------------------------------------------------------------

📊 Correlation với Electricity Consumption:
electricity_lag1              :  0.9823
electricity_rolling_mean_24h  :  0.9608
electricity_lag24             :  0.9

<Figure size 1000x1000 with 0 Axes>

In [7]:
# ============================================================================
# PHẦN 3: TRAIN XGBOOST MODEL VỚI PREDICTION INTERVALS
# ============================================================================

print("\n" + "=" * 80)
print("PHẦN 3: TRAIN XGBOOST MODEL VỚI PREDICTION INTERVALS")
print("=" * 80)

# Chuẩn bị dữ liệu cho training
np.random.seed(42)
train_buildings = np.random.choice(
    df_final['building_id'].unique(), 
    size=min(100, df_final['building_id'].nunique()), 
    replace=False
)

df_train = df_final[df_final['building_id'].isin(train_buildings)].copy()
df_train = df_train.sort_values(['building_id', 'timestamp'])

print(f"\nĐang train với {len(train_buildings)} buildings...")
print(f"Dataset shape: {df_train.shape}")

# Loại bỏ missing values trong target và các features quan trọng
df_train = df_train.dropna(subset=['electricity_consumption', 'sqm', 'airTemperature'])

# Fill missing values
for col in selected_continuous_features:
    if col in df_train.columns:
        df_train[col] = df_train.groupby('building_id')[col].transform(
            lambda x: x.fillna(x.median() if not x.isna().all() else 0)
        )

for col in selected_categorical_features:
    if col in df_train.columns:
        df_train[col] = df_train[col].fillna(
            df_train[col].mode()[0] if len(df_train[col].mode()) > 0 else 'Unknown'
        )

print(f"Sau khi xử lý missing values: {df_train.shape}")

# Tạo features và target
X = df_train[selected_continuous_features + selected_categorical_features].copy()
y = df_train['electricity_consumption'].copy()

# Encode categorical features
label_encoders_train = {}
for col in selected_categorical_features:
    if col in X.columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        label_encoders_train[col] = le

# Chia train/test theo thời gian (time series split)
df_train_sorted = df_train.sort_values('timestamp')
split_idx = int(len(df_train_sorted) * 0.8)

X_train = X.iloc[:split_idx]
y_train = y.iloc[:split_idx]
X_test = X.iloc[split_idx:]
y_test = y.iloc[split_idx:]

print(f"\nTrain set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTrain period: {df_train_sorted.iloc[0]['timestamp']} đến {df_train_sorted.iloc[split_idx-1]['timestamp']}")
print(f"Test period: {df_train_sorted.iloc[split_idx]['timestamp']} đến {df_train_sorted.iloc[-1]['timestamp']}")

# Train XGBoost model chính
print("\nĐang train XGBoost model...")

xgb_model = xgb.XGBRegressor(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    objective='reg:squarederror',
    eval_metric='rmse'
)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=50
)

# Predictions
y_pred_train = xgb_model.predict(X_train)
y_pred_test = xgb_model.predict(X_test)

# Metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"\n✅ Model Performance:")
print(f"\nTrain Set:")
print(f"  RMSE: {train_rmse:.2f} kWh")
print(f"  MAE:  {train_mae:.2f} kWh")
print(f"  R²:   {train_r2:.4f}")

print(f"\nTest Set:")
print(f"  RMSE: {test_rmse:.2f} kWh")
print(f"  MAE:  {test_mae:.2f} kWh")
print(f"  R²:   {test_r2:.4f}")

# Train models cho prediction intervals
print("\nĐang tính prediction intervals dựa trên residuals...")

# Tính residuals trên training set
residuals = y_train - y_pred_train

# Tính standard deviation của residuals
residual_std = np.std(residuals)

# Predictions trên test set
y_pred_mean = xgb_model.predict(X_test)

# Tạo prediction intervals (80% confidence interval)
confidence_level = 0.80
alpha = 1 - confidence_level
z_score = 1.28  # Cho 80% confidence interval

# Dựa trên percentile của residuals (chính xác hơn)
percentile_lower = (alpha / 2) * 100
percentile_upper = (1 - alpha / 2) * 100
residual_lower = np.percentile(residuals, percentile_lower)
residual_upper = np.percentile(residuals, percentile_upper)

y_pred_lower = y_pred_mean + residual_lower
y_pred_upper = y_pred_mean + residual_upper

print("✅ Đã tính prediction intervals")

# Tính coverage (tỷ lệ giá trị thực tế nằm trong interval)
coverage = np.mean((y_test >= y_pred_lower) & (y_test <= y_pred_upper))
print(f"\n📊 Prediction Interval Coverage: {coverage:.2%} (target: {confidence_level:.0%})")

# Tính độ rộng trung bình của interval
interval_width = np.mean(y_pred_upper - y_pred_lower)
print(f"📊 Average Interval Width: {interval_width:.2f} kWh")
print(f"📊 Residual Std: {residual_std:.2f} kWh")

# Tạo DataFrame kết quả dự đoán
print("\n" + "-" * 80)
print("Format Output - Dự đoán cho từng Tòa nhà")
print("-" * 80)

results_df = pd.DataFrame({
    'building_id': df_train_sorted.iloc[split_idx:]['building_id'].values,
    'timestamp': df_train_sorted.iloc[split_idx:]['timestamp'].values,
    'actual_consumption': y_test.values,
    'predicted_consumption': y_pred_mean,
    'predicted_lower': y_pred_lower,
    'predicted_upper': y_pred_upper,
    'prediction_interval_width': y_pred_upper - y_pred_lower
})

# Thêm thông tin về building
results_df = pd.merge(
    results_df,
    df_metadata[['building_id', 'primaryspaceusage', 'sqm', 'site_id']],
    on='building_id',
    how='left'
)

print(f"\n✅ Kết quả dự đoán cho {results_df['building_id'].nunique()} buildings")
print(f"   Tổng cộng {len(results_df)} predictions (mỗi prediction = 1 building × 1 giờ)")
print(f"\n📋 Format Output (10 dòng đầu):")
print(results_df.head(10))

print(f"\n📊 Thống kê:")
print(f"  - Số buildings: {results_df['building_id'].nunique()}")
print(f"  - Số timestamps: {results_df['timestamp'].nunique()}")
print(f"  - Số predictions: {len(results_df)}")
print(f"  - Trung bình prediction interval width: {results_df['prediction_interval_width'].mean():.2f} kWh")

# Lưu kết quả
results_df.to_csv('./output/predictions_results.csv', index=False)
print(f"\n✅ Đã lưu kết quả dự đoán vào: predictions_results.csv")

# Visualization: Predictions với intervals
print("\nĐang tạo visualizations...")

# Plot 1: Sample predictions cho một building
sample_building = train_buildings[0]
building_test = results_df[results_df['building_id'] == sample_building].head(100)

if len(building_test) > 0:
    fig, axes = plt.subplots(2, 1, figsize=(15, 10))
    
    timestamps = building_test['timestamp'].values
    actual = building_test['actual_consumption'].values
    predicted = building_test['predicted_consumption'].values
    lower = building_test['predicted_lower'].values
    upper = building_test['predicted_upper'].values
    
    axes[0].plot(timestamps, actual, 'b-', label='Actual', linewidth=2, marker='o', markersize=3)
    axes[0].plot(timestamps, predicted, 'r-', label='Predicted', linewidth=2, marker='s', markersize=3)
    axes[0].fill_between(timestamps, lower, upper, alpha=0.3, color='gray', label='80% Prediction Interval')
    axes[0].set_xlabel('Timestamp', fontsize=12)
    axes[0].set_ylabel('Electricity Consumption (kWh)', fontsize=12)
    axes[0].set_title(f'Predictions với Intervals - Building: {sample_building}', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True)
    plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=45)
    
    # Plot 2: Scatter plot - Actual vs Predicted
    sample_size_scatter = min(1000, len(y_test))
    sample_idx_scatter = np.random.choice(len(y_test), sample_size_scatter, replace=False)
    
    axes[1].scatter(y_test.iloc[sample_idx_scatter], y_pred_mean[sample_idx_scatter], alpha=0.5)
    axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2, label='Perfect Prediction')
    axes[1].set_xlabel('Actual Electricity Consumption (kWh)', fontsize=12)
    axes[1].set_ylabel('Predicted Electricity Consumption (kWh)', fontsize=12)
    axes[1].set_title('Actual vs Predicted', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.savefig('./output/predictions_visualization.png', dpi=150, bbox_inches='tight')
    print("✅ Đã lưu visualization vào: predictions_visualization.png")
    plt.close()


PHẦN 3: TRAIN XGBOOST MODEL VỚI PREDICTION INTERVALS

Đang train với 100 buildings...
Dataset shape: (1554730, 52)
Sau khi xử lý missing values: (1547157, 52)

Train set: 1237725 samples
Test set: 309432 samples

Train period: 2016-01-01 00:00:00 đến 2017-08-13 20:00:00
Test period: 2017-08-13 20:00:00 đến 2017-12-31 23:00:00

Đang train XGBoost model...
[0]	validation_0-rmse:352.01118	validation_1-rmse:123.34727
[50]	validation_0-rmse:132.06010	validation_1-rmse:35.51882
[100]	validation_0-rmse:64.43194	validation_1-rmse:36.27230
[150]	validation_0-rmse:37.55444	validation_1-rmse:36.95680
[199]	validation_0-rmse:24.55678	validation_1-rmse:37.77471

✅ Model Performance:

Train Set:
  RMSE: 24.56 kWh
  MAE:  5.30 kWh
  R²:   0.9954

Test Set:
  RMSE: 37.77 kWh
  MAE:  11.10 kWh
  R²:   0.9056

Đang tính prediction intervals dựa trên residuals...
✅ Đã tính prediction intervals

📊 Prediction Interval Coverage: 53.12% (target: 80%)
📊 Average Interval Width: 13.96 kWh
📊 Residual Std: 24.56

In [12]:
# ============================================================================
# PHẦN 4: SỬ DỤNG DiCE ĐỂ GỢI Ý ĐIỀU CHỈNH FEATURES
# ============================================================================

print("\n" + "=" * 80)
print("PHẦN 4: SỬ DỤNG DiCE ĐỂ GỢI Ý ĐIỀU CHỈNH FEATURES")
print("=" * 80)

# Chuẩn bị dữ liệu cho DiCE
# Chỉ lấy các features có thể điều chỉnh được
adjustable_features = [
    'sqm', 'occupants', 'yearbuilt', 'numberoffloors',
    'airTemperature', 'hour', 'day_of_week', 'month',
    'cloudCoverage', 'windSpeed'
]

adjustable_categorical = [
    'primaryspaceusage', 'site_id', 'timezone', 'season', 'is_weekend'
]

# Lấy các features có thể điều chỉnh (phải có trong df_train)
dice_features = [f for f in adjustable_features + adjustable_categorical if f in df_train.columns]

print(f"\n✅ Features cho DiCE: {dice_features}")
print(f"   Tổng cộng: {len(dice_features)} features")

# Train một model riêng cho DiCE chỉ với adjustable features
# (vì lag features và rolling features không thể điều chỉnh được)
print("\nĐang train model riêng cho DiCE (chỉ với adjustable features)...")

# Tạo dataset chỉ với adjustable features
X_dice = df_train[dice_features].copy()
y_dice = df_train['electricity_consumption'].copy()


PHẦN 4: SỬ DỤNG DiCE ĐỂ GỢI Ý ĐIỀU CHỈNH FEATURES

✅ Features cho DiCE: ['sqm', 'occupants', 'yearbuilt', 'numberoffloors', 'airTemperature', 'hour', 'day_of_week', 'month', 'cloudCoverage', 'windSpeed', 'primaryspaceusage', 'site_id', 'timezone', 'season', 'is_weekend']
   Tổng cộng: 15 features

Đang train model riêng cho DiCE (chỉ với adjustable features)...


In [13]:
X_dice.columns

Index(['sqm', 'occupants', 'yearbuilt', 'numberoffloors', 'airTemperature',
       'hour', 'day_of_week', 'month', 'cloudCoverage', 'windSpeed',
       'primaryspaceusage', 'site_id', 'timezone', 'season', 'is_weekend'],
      dtype='object')

In [16]:
# ============================================================================
# PHẦN 4: SỬ DỤNG DiCE ĐỂ GỢI Ý ĐIỀU CHỈNH FEATURES
# ============================================================================

print("\n" + "=" * 80)
print("PHẦN 4: SỬ DỤNG DiCE ĐỂ GỢI Ý ĐIỀU CHỈNH FEATURES")
print("=" * 80)

# Chuẩn bị dữ liệu cho DiCE
# Chỉ lấy các features có thể điều chỉnh được
adjustable_features = [
    'sqm', 'occupants', 'yearbuilt', 'numberoffloors',
    'airTemperature', 'hour', 'day_of_week', 'month',
    'cloudCoverage', 'windSpeed'
]

adjustable_categorical = [
    'primaryspaceusage', 'site_id', 'timezone', 'season', 'is_weekend'
]

# Lấy các features có thể điều chỉnh (phải có trong df_train)
dice_features = [f for f in adjustable_features + adjustable_categorical if f in df_train.columns]

print(f"\n✅ Features cho DiCE: {dice_features}")
print(f"   Tổng cộng: {len(dice_features)} features")

# Train một model riêng cho DiCE chỉ với adjustable features
# (vì lag features và rolling features không thể điều chỉnh được)
print("\nĐang train model riêng cho DiCE (chỉ với adjustable features)...")

# Tạo dataset chỉ với adjustable features
X_dice = df_train[dice_features].copy()
y_dice = df_train['electricity_consumption'].copy()

# Thêm building_id tạm thời để có thể groupby (nếu có trong df_train)
if 'building_id' in df_train.columns:
    X_dice['building_id'] = df_train['building_id'].values

# Fill missing values
for col in dice_features:
    if col in X_dice.columns:
        if col in adjustable_features:
            # Nếu có building_id, groupby theo building_id, nếu không thì fillna trực tiếp
            if 'building_id' in X_dice.columns:
                X_dice[col] = X_dice.groupby('building_id')[col].transform(
                    lambda x: x.fillna(x.median() if not x.isna().all() else 0)
                )
            else:
                X_dice[col] = X_dice[col].fillna(X_dice[col].median() if not X_dice[col].isna().all() else 0)
        else:
            X_dice[col] = X_dice[col].fillna(
                X_dice[col].mode()[0] if len(X_dice[col].mode()) > 0 else 'Unknown'
            )

# Xóa building_id khỏi X_dice trước khi train (nếu đã thêm)
if 'building_id' in X_dice.columns and 'building_id' not in dice_features:
    X_dice = X_dice.drop(columns=['building_id'])

# Lưu bản gốc với categorical features dưới dạng string (cho DiCE)
X_dice_original = X_dice.copy()
for col in adjustable_categorical:
    if col in X_dice_original.columns:
        X_dice_original[col] = X_dice_original[col].astype(str)

# Encode categorical features cho training
label_encoders_dice = {}
for col in adjustable_categorical:
    if col in X_dice.columns:
        le = LabelEncoder()
        X_dice[col] = le.fit_transform(X_dice[col].astype(str))
        label_encoders_dice[col] = le

# Chia train/test theo thời gian
df_train_sorted_dice = df_train.sort_values('timestamp')
split_idx_dice = int(len(df_train_sorted_dice) * 0.8)

X_train_dice = X_dice.iloc[:split_idx_dice]
y_train_dice = y_dice.iloc[:split_idx_dice]
X_test_dice = X_dice.iloc[split_idx_dice:]
y_test_dice = y_dice.iloc[split_idx_dice:]

# Train XGBoost model cho DiCE
xgb_model_dice = xgb.XGBRegressor(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    objective='reg:squarederror',
    eval_metric='rmse',
)

xgb_model_dice.fit(
    X_train_dice, y_train_dice,
    eval_set=[(X_train_dice, y_train_dice), (X_test_dice, y_test_dice)],
    verbose=50
)

# Đánh giá model DiCE
y_pred_dice = xgb_model_dice.predict(X_test_dice)
dice_rmse = np.sqrt(mean_squared_error(y_test_dice, y_pred_dice))
dice_r2 = r2_score(y_test_dice, y_pred_dice)

print(f"\n✅ Model DiCE Performance:")
print(f"  RMSE: {dice_rmse:.2f} kWh")
print(f"  R²:   {dice_r2:.4f}")

# Tạo wrapper class cho model để tự động encode categorical features
class XGBoostWrapper:
    """Wrapper class để tự động encode categorical features trước khi predict"""
    def __init__(self, model, label_encoders, categorical_features):
        self.model = model
        self.label_encoders = label_encoders
        self.categorical_features = categorical_features
    
    def predict(self, X):
        """Predict với tự động encode categorical features"""
        X_encoded = X.copy()
        
        # Encode categorical features
        for col in self.categorical_features:
            if col in X_encoded.columns:
                if col in self.label_encoders:
                    le = self.label_encoders[col]
                    # Chuyển đổi về string và encode
                    X_encoded[col] = X_encoded[col].astype(str)
                    # Xử lý các giá trị chưa thấy (unknown values)
                    X_encoded[col] = X_encoded[col].apply(
                        lambda x: x if x in le.classes_ else le.classes_[0]
                    )
                    X_encoded[col] = le.transform(X_encoded[col])
                else:
                    # Nếu không có encoder, giữ nguyên (có thể là integer rồi)
                    if X_encoded[col].dtype == 'object':
                        # Nếu là object nhưng không có encoder, chuyển về 0
                        X_encoded[col] = 0
        
        # Đảm bảo tất cả columns là numeric
        for col in X_encoded.columns:
            if X_encoded[col].dtype == 'object':
                X_encoded[col] = pd.to_numeric(X_encoded[col], errors='coerce').fillna(0)
        
        return self.model.predict(X_encoded)

# Tạo wrapped model
xgb_model_wrapped = XGBoostWrapper(
    xgb_model_dice,
    label_encoders_dice,
    adjustable_categorical
)

# Tạo dataframe cho DiCE (sử dụng bản gốc với categorical features dưới dạng string)
X_train_dice_original = X_dice_original.iloc[:split_idx_dice]
df_for_dice = pd.concat([X_train_dice_original, y_train_dice], axis=1)
df_for_dice = df_for_dice.reset_index(drop=True)

print(f"\n✅ Dataset cho DiCE: {df_for_dice.shape}")
print(f"   Features: {list(df_for_dice.columns[:-1])}")
print(f"   Target: electricity_consumption")

# Tạo DiCE Data object
dice_data = dice_ml.Data(
    dataframe=df_for_dice,
    continuous_features=[f for f in adjustable_features if f in df_for_dice.columns],
    outcome_name='electricity_consumption'
)

print("\n✅ DiCE Data object đã được tạo!")

# Tạo DiCE Model object (sử dụng wrapped model)
dice_model = dice_ml.Model(
    model=xgb_model_wrapped,  # Sử dụng wrapped model để tự động encode
    backend='sklearn',
    model_type='regressor'
)

print("✅ DiCE Model object đã được tạo!")

# Tạo DiCE Explainer
explainer = Dice(dice_data, dice_model, method='random')  # Có thể đổi thành 'genetic' nếu muốn kết quả tốt hơn

print("✅ DiCE Explainer đã được tạo với method='random'!")
print("   (Có thể đổi thành 'genetic' nếu muốn kết quả tốt hơn nhưng chậm hơn)")

# Chọn một building để tạo counterfactual explanations
# Lấy một building có lượng điện tiêu thụ cao từ training data
high_consumption_idx_dice = y_train_dice.idxmax()

# Tạo query_instance cho DiCE (sử dụng bản gốc chưa encode, với categorical dưới dạng string)
query_instance_dice = X_train_dice_original.loc[[high_consumption_idx_dice]][dice_features].copy()

# Lấy thông tin building từ df_train
building_info = df_train_sorted_dice.loc[high_consumption_idx_dice]

print("\n" + "-" * 80)
print("Building được chọn để tối ưu:")
print("-" * 80)
print(f"  Building ID: {building_info.get('building_id', 'N/A')}")
print(f"  Lượng điện tiêu thụ thực tế: {y_train_dice.loc[high_consumption_idx_dice]:.2f} kWh")
print(f"\nFeatures hiện tại (adjustable features):")
for feature in dice_features:
    if feature in query_instance_dice.columns:
        val = query_instance_dice[feature].iloc[0]
        print(f"  {feature:25s}: {val}")

# Dự đoán lượng điện với model DiCE (chỉ với adjustable features)
current_prediction = xgb_model_wrapped.predict(query_instance_dice[dice_features])[0]
print(f"\n  Dự đoán từ model DiCE: {current_prediction:.2f} kWh")

# Định nghĩa threshold
THRESHOLD = np.percentile(y_dice, 50)  # 50th percentile (median) của y_dice
# Hoặc có thể đặt thủ công: THRESHOLD = 500

print(f"\n📊 Ngưỡng tối đa cho phép: {THRESHOLD:.2f} kWh")
print(f"📊 Lượng điện hiện tại: {current_prediction:.2f} kWh")

if current_prediction > THRESHOLD:
    print(f"⚠️  Lượng điện hiện tại VƯỢT QUÁ ngưỡng! Cần điều chỉnh features.")
    reduction_needed = current_prediction - THRESHOLD
    reduction_pct = (reduction_needed / current_prediction) * 100
    print(f"   Cần giảm: {reduction_needed:.2f} kWh ({reduction_pct:.1f}%)")
else:
    print(f"✅ Lượng điện hiện tại trong ngưỡng cho phép.")

# Tạo counterfactual explanations
print("\n" + "-" * 80)
print("Đang tạo counterfactual explanations...")
print("(Quá trình này có thể mất vài phút...)")
print("-" * 80)

# Xác định permitted_range dựa trên dữ liệu thực tế
permitted_range = {}
for col in adjustable_features:
    if col in df_for_dice.columns:
        col_min = df_for_dice[col].min()
        col_max = df_for_dice[col].max()
        # Mở rộng một chút để có nhiều lựa chọn hơn
        col_range = col_max - col_min
        permitted_range[col] = [max(0, col_min - 0.1 * col_range), col_max + 0.1 * col_range]

print(f"\nPermitted ranges cho continuous features:")
for col, (min_val, max_val) in permitted_range.items():
    print(f"  {col:25s}: [{min_val:10.2f}, {max_val:10.2f}]")

# Tạo counterfactuals
# Sử dụng query_instance_dice (chỉ có adjustable features) cho DiCE
try:
    counterfactuals = explainer.generate_counterfactuals(
        query_instance_dice,  # Sử dụng query_instance_dice thay vì query_instance
        total_CFs=5,  # Số lượng counterfactual examples
        desired_range=[0, THRESHOLD],  # Khoảng giá trị mong muốn
        permitted_range=permitted_range,  # Giới hạn phạm vi thay đổi
    )
    print("\n✅ Đã tạo xong counterfactual explanations!")
    
    # Hiển thị kết quả
    print("\n" + "=" * 80)
    print("COUNTERFACTUAL EXPLANATIONS - Các phương án điều chỉnh features")
    print("=" * 80)
    
    print("\n📊 Instance gốc (hiện tại):")
    print(f"  Lượng điện dự đoán: {current_prediction:.2f} kWh")
    print(f"  Features (adjustable):")
    for feature in dice_features:
        if feature in query_instance_dice.columns:
            print(f"    {feature}: {query_instance_dice[feature].iloc[0]}")
    
    print("\n" + "-" * 80)
    print("Các phương án đề xuất (Counterfactuals):")
    print("-" * 80)
    
    # Hiển thị counterfactuals
    cf_df = counterfactuals.visualize_as_dataframe(show_only_changes=True)
    print(cf_df)
    
    # Phân tích chi tiết các phương án
    print("\n" + "=" * 80)
    print("PHÂN TÍCH CHI TIẾT CÁC PHƯƠNG ÁN")
    print("=" * 80)
    
    # Lấy counterfactual examples
    cf_examples = counterfactuals.cf_examples_list[0].final_cfs_list
    
    for i, cf in enumerate(cf_examples, 1):
        # Chuyển đổi về DataFrame
        cf_df_single = pd.DataFrame([cf], columns=dice_features)
        
        # Dự đoán lượng điện cho counterfactual này (sử dụng wrapped model)
        cf_prediction = xgb_model_wrapped.predict(cf_df_single[dice_features])[0]
        
        print(f"\n--- Phương án {i} ---")
        print(f"📊 Dự đoán lượng điện: {cf_prediction:.2f} kWh (mục tiêu: <= {THRESHOLD:.2f} kWh)")
        
        if cf_prediction <= THRESHOLD:
            print("  ✅ Đạt mục tiêu!")
        else:
            print("  ⚠️  Chưa đạt mục tiêu")
        
        # So sánh với instance gốc
        print("\n  Thay đổi so với hiện tại:")
        for col in dice_features:
            if col in query_instance_dice.columns and col in cf_df_single.columns:
                original_val = query_instance_dice[col].iloc[0]
                new_val = cf_df_single[col].iloc[0]
                
                if pd.isna(original_val) or pd.isna(new_val):
                    continue
                
                # Xử lý cho cả số và categorical
                if isinstance(original_val, (int, float)) and isinstance(new_val, (int, float)):
                    if abs(original_val - new_val) > 0.01:  # Có thay đổi đáng kể
                        change = new_val - original_val
                        change_pct = (change / original_val * 100) if original_val != 0 else 0
                        print(f"    {col:25s}: {original_val:10.2f} → {new_val:10.2f} (thay đổi: {change:+.2f}, {change_pct:+.1f}%)")
                else:
                    if original_val != new_val:
                        print(f"    {col:25s}: {original_val} → {new_val}")
        
        reduction = current_prediction - cf_prediction
        reduction_pct = (reduction / current_prediction * 100) if current_prediction > 0 else 0
        print(f"\n  ✅ Giảm lượng điện: {reduction:.2f} kWh ({reduction_pct:.1f}%)")
    
    # Lưu counterfactuals
    cf_df.to_csv('/root/projects/cmcproject/grid_balancer/reverse_LLM/counterfactuals_results.csv', index=False)
    print(f"\n✅ Đã lưu counterfactuals vào: counterfactuals_results.csv")
    
except Exception as e:
    print(f"\n⚠️  Lỗi khi tạo counterfactuals: {e}")
    print("   Có thể do XGBoost model không tương thích hoàn toàn với DiCE.")
    print("   Hãy thử sử dụng method='genetic' hoặc kiểm tra lại model.")
    import traceback
    traceback.print_exc()


PHẦN 4: SỬ DỤNG DiCE ĐỂ GỢI Ý ĐIỀU CHỈNH FEATURES

✅ Features cho DiCE: ['sqm', 'occupants', 'yearbuilt', 'numberoffloors', 'airTemperature', 'hour', 'day_of_week', 'month', 'cloudCoverage', 'windSpeed', 'primaryspaceusage', 'site_id', 'timezone', 'season', 'is_weekend']
   Tổng cộng: 15 features

Đang train model riêng cho DiCE (chỉ với adjustable features)...
[0]	validation_0-rmse:354.38040	validation_1-rmse:125.54885
[50]	validation_0-rmse:205.63736	validation_1-rmse:375.48133
[100]	validation_0-rmse:167.64600	validation_1-rmse:395.42209
[150]	validation_0-rmse:143.45897	validation_1-rmse:396.85885
[199]	validation_0-rmse:123.80094	validation_1-rmse:397.29160

✅ Model DiCE Performance:
  RMSE: 397.29 kWh
  R²:   -9.4425

✅ Dataset cho DiCE: (1237725, 16)
   Features: ['sqm', 'occupants', 'yearbuilt', 'numberoffloors', 'airTemperature', 'hour', 'day_of_week', 'month', 'cloudCoverage', 'windSpeed', 'primaryspaceusage', 'site_id', 'timezone', 'season', 'is_weekend']
   Target: electri

100%|██████████| 1/1 [00:00<00:00,  1.50it/s]


✅ Đã tạo xong counterfactual explanations!

COUNTERFACTUAL EXPLANATIONS - Các phương án điều chỉnh features

📊 Instance gốc (hiện tại):
  Lượng điện dự đoán: 134542.52 kWh
  Features (adjustable):
    sqm: 39822.6
    occupants: 0.0
    yearbuilt: 0.0
    numberoffloors: 0.0
    airTemperature: 30.6
    hour: 15
    day_of_week: 2
    month: 9
    cloudCoverage: 0.0
    windSpeed: 3.6
    primaryspaceusage: Education
    site_id: Bull
    timezone: US/Central
    season: Fall
    is_weekend: 0

--------------------------------------------------------------------------------
Các phương án đề xuất (Counterfactuals):
--------------------------------------------------------------------------------
Query instance (original outcome : 134543.0)


,sqm,occupants,yearbuilt,numberoffloors,airTemperature,hour,day_of_week,month,cloudCoverage,windSpeed,primaryspaceusage,site_id,timezone,season,is_weekend,electricity_consumption
0,39822.601562,0.0,0.0,0.0,30.6,15,2,9,0.0,3.6,Education,Bull,US/Central,Fall,0,134543.0



Diverse Counterfactual set (new outcome: [0, np.float64(73.36)])


,sqm,occupants,yearbuilt,numberoffloors,airTemperature,hour,day_of_week,month,cloudCoverage,windSpeed,primaryspaceusage,site_id,timezone,season,is_weekend,electricity_consumption
0,-,36.5,-,-,34.5,-,-,-,-,21.2,Parking,-,-,-,-,-
1,-,-,356.8,-,46.3,-,-,-,-,-,-,-,-,Spring,-,-
2,-,552.5,-,-,3.0,-,-,0,-,-,Entertainment/public assembly,-,Europe/London,-,-,-
3,-,-,356.8,-,46.3,-,-,-,-,-,-,-,-,-,-,-
4,-,-,-,-,20.9,21,-,-,-,-,Office,Peacock,-,-,-,-


None

PHÂN TÍCH CHI TIẾT CÁC PHƯƠNG ÁN

⚠️  Lỗi khi tạo counterfactuals: 'NoneType' object is not iterable
   Có thể do XGBoost model không tương thích hoàn toàn với DiCE.
   Hãy thử sử dụng method='genetic' hoặc kiểm tra lại model.


Traceback (most recent call last):
  File "/var/folders/9j/40pxvw_12zl9rw6mldc3vvwr0000gn/T/ipykernel_91245/3958352869.py", line 286, in <module>
    for i, cf in enumerate(cf_examples, 1):
                 ~~~~~~~~~^^^^^^^^^^^^^^^^
TypeError: 'NoneType' object is not iterable
